In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# update working directory
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom')

In [ ]:
!pip install tensorboardX
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitli

In [ ]:
import os
import time
import torch.multiprocessing
from copy import deepcopy
from tensorboardX import SummaryWriter
from tqdm import tqdm
import shutil

from parameters import Parameters
from train.trainer import Trainer
from deepfake_datasets.datasets import get_dataloader
from train.early_stopping import EarlyStopping
from train.validate import validate_detection, validate_fully_supervised_localization
from utils.utils import compute_mean_iou, compute_mean_ap, compute_mean_f1, compute_mean_acc_detection, compute_mean_ap_detection, compute_batch_iou, compute_batch_localization_f1, compute_batch_ap, compute_accuracy_detection, compute_average_precision_detection


In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
new_dataset_name = 'train_all_4_datasets'

zip_path = f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets_zip/combined_training_for_ood/{new_dataset_name}.zip'
new_content_root_path = f'/content/datasets/dolos_data/celebahq/fake/'

os.makedirs(new_content_root_path, exist_ok=True)

# move the dataset from drive to /content (SSD) for better performance in I/O
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(new_content_root_path)

In [ ]:
# here we will only train on a third of each dataset

# def keep_only_chosen_dataset(folder_path, dataset_name):
#     for file_name in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, file_name)
#         if os.path.isfile(file_path) and dataset_name not in file_name:
#             os.remove(file_path)

# chosen_train_dataset = 'ldm'
# base_path = '/content/datasets/dolos_data/celebahq/fake/train_all_4_datasets'
# subfolders = [
#     'images/train', 'images/valid', 'images/test',
#     'masks/train', 'masks/valid', 'masks/test'
# ]

# # keep only the images of this specific dataset
# for subfolder in subfolders:
#     full_path = os.path.join(base_path, subfolder)
#     if os.path.exists(full_path):
#         keep_only_chosen_dataset(full_path, chosen_train_dataset)

In [ ]:
# set parameters
params = Parameters()

# set experiment name
params.experiment_name = 'train_all_4_limited_samples'

# set backbone
params.arch = 'CLIP:ViT-L/14,RN50'

# create output dirs
params.create_output_dirs()

# uncomment lines below to set new datasets
# new_drive_root_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets/dolos_data/celebahq/'
# new_root_path = '/content/dataset'
# new_root_path = '..\\DeCLIP-main\\datasets\\dolos_data\\celebahq\\'
# new_root_path = 'D:\\Python\\DeCLIP-main\\datasets\\dolos_data\\celebahq\\'


# move the dataset from drive to /content for better performance in I/O
# shutil.copytree(new_drive_root_path, new_root_path, dirs_exist_ok=True)

new_root_path = '/content/datasets/dolos_data/celebahq/'
new_dataset_name = 'train_all_4_datasets'

params.update_dolos_data_paths(new_root_path, new_dataset_name)

# fix the backbone - to train only the decoder
params.fix_backbone = True

# set the feature layer and the decoder type
params.feature_layer = 'layer20'
params.decoder_type = 'conv-12'

# set the batch size and num threads
params.batch_size = 32
params.num_threads = 8

# set the number of epochs
params.num_iter = 30

# set loss type
params.loss_type = 'bce'

for key, value in params.__dict__.items():
    print(f"{key}: {value}")

experiment_name: train_all_4_limited_samples
task_type: fully_supervised_localization
data_label: train
arch: CLIP:ViT-L/14,RN50
fix_backbone: True
weight_decay: 0.0
batch_size: 32
num_threads: 8
init_type: normal
init_gain: 0.02
train_dataset: train_all_4_datasets
decoder_type: conv-12
feature_layer: layer20
early_stop_epochs: 5
optim: adam
beta1: 0.9
lr: 0.001
show_loss_freq: 50
num_iter: 30
data_root_path: /content/datasets/dolos_data/celebahq
train_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/train
valid_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/valid
test_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/test
train_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/masks/train
valid_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/masks/valid
test_masks_ground_truth_path: /content/datasets/dolos_d

In [ ]:
# get the model, along with its trainer
model_trainer = Trainer(params)

# state_dict = torch.load('/content/experiments/training_inital/models/best_localization_model_iou_70.8967.pth', map_location='cpu')
# model_trainer.model.load_state_dict(state_dict['model'], strict=False)
# model_trainer.model = model_trainer.model.cuda()

# model_trainer.optimizer.load_state_dict(state_dict['optimizer'])
# model_trainer.total_steps = state_dict['total_steps']

100%|███████████████████████████████████████| 890M/890M [00:28<00:00, 32.9MiB/s]
100%|███████████████████████████████████████| 244M/244M [00:04<00:00, 55.2MiB/s]


In [ ]:
# get the dataloaders
train_loader = get_dataloader(params)

val_params = deepcopy(params)
val_params.data_label = 'valid'
val_loader = get_dataloader(val_params)

In [ ]:
num_samples_ldm = 1000 # we can go even more, with 1750 or 3000, it keeps the rest stable it seems
num_samples_repaint = 500
num_samples_lama = 500
num_samples_pluralistic = 500

for file_path in train_loader.dataset.fake_images:
    if 'ldm' in file_path and num_samples_ldm > 0:
        num_samples_ldm -= 1
        continue
    elif 'repaint' in file_path and num_samples_repaint > 0:
        num_samples_repaint -= 1
        continue
    elif 'lama' in file_path and num_samples_lama > 0:
        num_samples_lama -= 1
        continue
    elif 'pluralistic' in file_path and num_samples_pluralistic > 0:
        num_samples_pluralistic -= 1
        continue
    else:
        os.remove(file_path) # delete the rest

train_loader = get_dataloader(params) # on the new, limited dataset

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# start the training loop
early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.001)
best_metric = 0.0 # iou for localization, ap for detection
print('Length of training set:', len(train_loader.dataset))
print('Length of validation set:', len(val_loader.dataset))
start_time = time.time()
for epoch in tqdm(range(params.num_iter)):
    print('Epoch:', epoch)

    epoch_loss = 0
    for data in train_loader:
        model_trainer.total_steps += 1

        model_trainer.set_input(data)
        model_trainer.optimize_parameters()

        if model_trainer.total_steps % params.show_loss_freq == 0:
            epoch_loss += model_trainer.loss.item()
            print(f'Train Loss: {model_trainer.loss.item():.4f} at step {model_trainer.total_steps} \t Iter time: {(time.time() - start_time) / model_trainer.total_steps:.2f}')

    epoch_loss /= len(train_loader.dataset)
    print(f'Average Train Loss: {epoch_loss:.4f}')

    # compute training metrics
    if params.task_type == 'fully_supervised_localization':
        compute_mean_iou(model_trainer.ious, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.ious = []

        compute_mean_f1(model_trainer.f1_best, model_trainer.f1_fixed, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.f1_best = []
        model_trainer.f1_fixed = []

        compute_mean_ap(model_trainer.ap, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.ap = []


    # validate the model
    print('Validation:')
    if params.task_type == 'fully_supervised_localization':
        ious, f1_best, f1_fixed, mean_ap, _ = validate_fully_supervised_localization(model_trainer.model, val_loader, params.train_dataset)

        # compute metrics
        mean_iou = compute_mean_iou(ious, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        mean_ap = compute_mean_ap(mean_ap, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        # save the model if the mean iou is improved
        if mean_iou > best_metric:
            best_metric = mean_iou
            model_trainer.save_model(f'best_localization_model_iou_{mean_iou:.4f}.pth')
            print(f'Best model saved at epoch {epoch}!')

        # check for early stopping
        early_stopping(mean_iou)

    # check if early stopping is triggered
    if early_stopping.early_stop:
        print("Early stopping triggered")
        continue_training = model_trainer.adjust_learning_rate()
        if continue_training:
            print("Continuing training with a learning rate reduced by a factor of 10")
            early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.002) # adjust the delta only once, otherwise stop completely
        else:
            print(f"Early stopping training at epoch {epoch}")
            break
    print()
    torch.cuda.empty_cache()

Length of training set: 2500
Length of validation set: 1200


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0
Train Loss: 0.3152 at step 50 	 Iter time: 1.34
Average Train Loss: 0.0001
Train epoch 0 Mean IOU: 0.0
Train epoch 0 Mean F1 best: 0.5976
Train epoch 0 Mean F1 fixed: 0.1052
Train epoch 0 Mean AP: 0.4639
Validation:
Length of dataset:  1200



100%|██████████| 38/38 [00:23<00:00,  1.64it/s]


Validation at epoch 0 Mean IOU: 0.0
Validation at epoch 0 Mean F1 best: 0.5722
Validation at epoch 0 Mean F1 fixed: 0.09
Validation at epoch 0 Mean AP: 0.4727


...

Best model saved at epoch 27!

Epoch: 28
Train Loss: 0.0878 at step 2250 	 Iter time: 1.56
Average Train Loss: 0.0000
Train epoch 28 Mean IOU: 78.74
Train epoch 28 Mean F1 best: 0.9411
Train epoch 28 Mean F1 fixed: 0.9062
Train epoch 28 Mean AP: 0.9169
Validation:
Length of dataset:  1200



 97%|█████████▋| 29/30 [59:44<02:03, 123.17s/it]

Validation at epoch 28 Mean IOU: 54.44
Validation at epoch 28 Mean F1 best: 0.7384
Validation at epoch 28 Mean F1 fixed: 0.66
Validation at epoch 28 Mean AP: 0.7309
Early Stopping counter: 1 out of 5

Epoch: 29
Train Loss: 0.0872 at step 2300 	 Iter time: 1.56
Train Loss: 0.0844 at step 2350 	 Iter time: 1.56
Average Train Loss: 0.0001
Train epoch 29 Mean IOU: 78.23
Train epoch 29 Mean F1 best: 0.9393
Train epoch 29 Mean F1 fixed: 0.8986
Train epoch 29 Mean AP: 0.9131
Validation:
Length of dataset:  1200



100%|██████████| 30/30 [1:01:46<00:00, 123.56s/it]

Validation at epoch 29 Mean IOU: 54.37
Validation at epoch 29 Mean F1 best: 0.7388
Validation at epoch 29 Mean F1 fixed: 0.6594
Validation at epoch 29 Mean AP: 0.7299
Early Stopping counter: 2 out of 5



In [ ]:
model_dir = '/content/experiments/train_all_4_limited_samples/models/'
model_files = sorted(
    [f for f in os.listdir(model_dir) if f.endswith('.pth')]
)

best_model = model_files[-1]
best_model_path = os.path.join(model_dir, best_model)

In [ ]:
shutil.copy(best_model_path, f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/{os.path.basename(best_model_path)}')

'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/best_localization_model_iou_54.8946.pth'